In [4]:
import speech_recognition as sr
r = sr.Recognizer()
mic = sr.Microphone()
with mic as source:
    audio = r.listen(source)
    text=r.recognize_google(audio)
    print ("you said: " , text) 
    

UnknownValueError: 

In [1]:
import speech_recognition as sr # for speech recognition
#Selenium sends the standard Python commands to different browsers,despite the variation in browser's design.
from selenium import webdriver
from getpass import getpass
from selenium import webdriver
from selenium.webdriver.common.keys import Keys #The Keys class provide keys in the keyboard like RETURN, F1, ALT, ENTER etc
from selenium.common.exceptions import NoSuchElementException
import mysql.connector #for database connectivity
from mysql.connector import MySQLConnection, Error #internally used by Connector/Python to raise MySQL client and server errors
def voice_recognition():
    word=""
    flag=0
    while word=="" and flag<3:
        r = sr.Recognizer() # a Recognizer instance is to recognize speech
        mic = sr.Microphone() # use the default microphone as the audio source
        with mic as source:
             r.adjust_for_ambient_noise(source) #ambient noise suppression/adjustment
             print ("Search by title of book\nNow listening... ")
             #listens for the user's input
             audio = r.listen(source) #listen() method takes an audio source as its first argument and records input from the source until silence is detected.
             try:
                 word = r.recognize_google(audio) # Recognizing Speech in the Audio
                 print ("you said: " , word)
                 verify=input("Correct or not:Yes/No\n")
                 if(verify=='no' or verify=='No'):
                        print("Try Again!")
                        flag=flag+1
                        word=""
             except sr.UnknownValueError: #error occurs when google could not understand what was said
                print("Google Speech Recognition could not understand audio")
                flag=flag+1
             except sr.RequestError as e:
                print("Could not request results from Google Speech Recognition service; {0}".format(e))
                flag=flag+1
    if(word==""):
        word=input("Enter name of the book you want to search:")
    print(word)
    return word
conn=mysql.connector.connect(host="localhost",user="root",passwd="samuk",database="sih2020")
#method sets up a connection, establishing a session with the MySQL server. If no arguments are given, it uses default values
user=input("Login As:1.Student\t2.Admin\n")
if(user=='1'):
    choice=input("Do you want to search books:Yes/No\n")
    while choice=='Yes' or choice=='yes' :
        word=voice_recognition()
        w=(word,)
        query_exist="""SELECT EXISTS(select * from Lib where title = %s)"""
        mycursor_exist = conn.cursor(buffered=True)#MySQLCursorBuffered is useful in situations where multiple queries, with small result sets
        mycursor_exist.execute(query_exist,w)#This method executes the given database operation (query or command)
        if(mycursor_exist.fetchone()[0]): #retrieves next row of a query result set and returns a single sequence, or None if no more rows are available
            query = """SELECT * from Lib WHERE title = %s """
            mycursor = conn.cursor(buffered=True)
            mycursor.execute(query,w)
            myresult = mycursor.fetchall()
            for x in myresult:
                print(x)
            conn.commit() #sends a COMMIT statement to the MySQL server, committing the current transaction
            mycursor.close()
            mycursor_exist.close()
        else:
            print("Record not found")
        choice=input("Do you want to search more books: Yes /No\n")
    else:
            print("Thank You!")
else:
    count=0
    while count<3:
        password = getpass("Enter password:") #getpass method used to get input password and to hide it's text
        if(password=="admin123"): #admin authentication
            count=5
            choice=input("Do you want to search books: Yes/No\n")
            while choice=='Yes' or choice=='yes':
                word=voice_recognition()
                w=(word,)
                query_exist="""SELECT EXISTS(select * from Lib where title = %s)"""
                mycursor_exist = conn.cursor(buffered=True)
                mycursor_exist.execute(query_exist,w)
                try:
                    driver = webdriver.Chrome(executable_path="C:\\Users\\HP\\AppData\\Local\\Programs\\Python\\Python37-32\\Scripts\\chromedriver.exe")#Chromedeiver path
                    driver.get("https://www.goodreads.com")#get() is used to navigate particular URL(website) and wait till page load
                    driver.find_element_by_name("query").send_keys(word)#finding web element and entering word(title) into search block
                    driver.find_element_by_name("query").send_keys(Keys.ENTER)# Enter key
                    driver.find_element_by_class_name("bookTitle").send_keys(Keys.ENTER)# first record from the result page is fetched
                    genre1=driver.find_element_by_xpath('//a[@class="actionLinkLite bookPageGenreLink"]')# locating the genre
                    aut=driver.find_element_by_xpath('//span[@itemprop="name"]')# locating the author
                    #converting list to text
                    g=genre1.text
                    a=aut.text
                    if(mycursor_exist.fetchone()[0]): #if book already in the database only update the genre and other details
                        query = """ UPDATE Lib SET genre = %s WHERE author = %s """
                        data = (g,a)
                        mycursor_admin = conn.cursor(buffered=True)
                        mycursor_admin.execute(query,data)    
                    else:# add book to the database since not present
                        copies=input("No record found in the database\nadding book to the database\nEnter number of copies:\n")
                        data_tuple=(word,a,int(copies),g)
                        # prepare query and data
                        insert_query="""insert into Lib(title,author,no_of_copies,genre) values (%s,%s,%s,%s)"""
                        mycursor_admin = conn.cursor(buffered=True)
                        mycursor_admin.execute(insert_query,data_tuple)
                        mycursor_admin.close()#close the cursor
                    query_display = """SELECT * from Lib where title=%s """
                    mycursor_admin1 = conn.cursor(buffered=True)
                    mycursor_admin1.execute(query_display,w)
                    myresult_admin= mycursor_admin1.fetchall()#fetch all rows from resultset returned by query
                    for x in myresult_admin:
                        print(x)
                    mycursor_admin1.close()#close the cursor
                    mycursor_exist.close()#close the cursor
                    conn.commit()#commit transaction
                    driver.close()#closes the browser window which is currently in focus
                except NoSuchElementException:
                    print("Sorry, Details not found!Try for another book.")
                choice=input("Do you want to search more books: Yes /No\n")
            else:
                print("Thank you!")
        else: #Admin verification failure
            print("User Authentication Failed!\tTry Again!")
            count=count+1
    if(count==3):
        print("\nToo many unsuccessful attempts!")
gen=input("Do you want to see categorization:Yes or No:\n")
if(gen=='Yes' or gen=='yes'):
    genre_to_find=input("Enter genre name you want to search: ")
    gtf=(genre_to_find,)
    query_to_genre="""select * from Lib where genre=%s""" #to display categorization according to genre

    genre_cursor_to_search=conn.cursor(buffered=True)
    genre_cursor_to_search.execute(query_to_genre,gtf)
    categorization= genre_cursor_to_search.fetchall()
    for x in categorization:
                print(x)
else:
    print("Thank you!")
conn.close() #close the mysql connection


ModuleNotFoundError: No module named 'selenium'

In [31]:
100101import mysql.connector
mydb=mysql.connector.connect(host="localhost",user="root",passwd="root123",database="newdb")
mycursor=mydb.cursor()
mycursor.execute("describe abc")
for i in mycursor:
    print(i)
    
    

('name', 'varchar(20)', 'YES', '', None, '')
('roll', 'int(11)', 'YES', '', None, '')


In [25]:
import mysql.connector
mydb=mysql.connector.connect(host="lochalhost",user="root",passwd="root123",port=3306)

InterfaceError: 2003: Can't connect to MySQL server on 'lochalhost:3306' (11001 getaddrinfo failed)